In [1]:
import loader as l
import plots as pl
import analysis as a
import patterns as pt
import pandas as pd

WARNING (pytensor.tensor.blas): Using NumPy C-API based implementation for BLAS functions.


In [3]:
tickers = 'NKE'

In [4]:
stock_data = l.load_data(tickers)
stock_data = l.check_clean_data(stock_data, verbose=True)

From NKE dataset were removed 6 rows with missing or incorrect data


In [5]:
stock_data[tickers]['splits']

Date
1995-10-31 00:00:00-05:00    2.0
1996-10-24 00:00:00-04:00    2.0
2007-04-03 00:00:00-04:00    2.0
2012-12-26 00:00:00-05:00    2.0
2015-12-24 00:00:00-05:00    2.0
Name: Stock Splits, dtype: float64

In [6]:
df = stock_data[tickers]['historical_data']
df

,Open,High,Low,Close,Volume,Dividends,Stock Splits
Date,,,,,,,
1995-01-03 00:00:00-05:00,1.744,1.744,1.697,1.697,21308800,0.0,0.0
1995-01-04 00:00:00-05:00,1.697,1.708,1.685,1.694,12208000,0.0,0.0
1995-01-05 00:00:00-05:00,1.691,1.708,1.691,1.694,7913600,0.0,0.0
1995-01-06 00:00:00-05:00,1.688,1.703,1.667,1.682,8915200,0.0,0.0
1995-01-09 00:00:00-05:00,1.676,1.697,1.676,1.694,3152000,0.0,0.0
...,...,...,...,...,...,...,...
2024-06-28 00:00:00-04:00,77.130,79.050,74.550,75.370,129967000,0.0,0.0
2024-07-01 00:00:00-04:00,75.500,77.060,74.290,76.830,44369000,0.0,0.0
2024-07-02 00:00:00-04:00,76.820,76.940,75.390,76.040,26183400,0.0,0.0


In [7]:
pl.plot_chart(df, ticker=tickers)

In [8]:
support_df = a.calculate_ATR(df)

/Users/carlonestor/Documents/GitHub/statistical_analysis_candlestick_patterns/analysis.py:48: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy



In [9]:
pattern_name = "Bullish Marubozu"  # replace with the desired pattern name

# check if the pattern exists in the patterns dictionary
if pattern_name in pt.patterns:
    pattern_function = pt.patterns[pattern_name]['function']
    # use the function to obtain the mask of the pattern dates
    mask = pattern_function(pd.concat([df, support_df], axis=1))
    # filter the mask to ensure consistency in the future analysis
    mask = pt.filter_mask(mask, step=10)
    print(mask.sum())
else:
    print(f"The pattern '{pattern_name}' does not exist.")

86


In [10]:
pl.plot_close_with_patterns(df, tickers, mask)

In [11]:
pl.plot_patterns(df, mask, num_candles=pt.patterns[pattern_name]['candles'], ticker=tickers, pattern_name=pattern_name)

In [12]:
returns = a.calculate_cumReturns_periods(df, mask)
log_returns = a.calculate_log_cumReturns_periods(df, mask)
# round the returns to 3 decimal places and multiply by 100 to make percentages more readable
rounded_returns = {k: [round(100*r,3) for r in v] for k, v in returns.items()}
print(returns.keys())
print(len(returns[1]))
print(returns[1])

dict_keys([1, 2, 3, 4, 5, 6, 7, 8, 9, 10])
86
[-0.01558891454965353, -0.013832853025936611, -0.0016713091922004968, -0.002967359050444997, 0.008399440037330957, 0.011932078935291532, -0.02273603082851644, -0.006322052807735182, -0.005446293494705064, -0.029779630732578943, 0.004691005506832669, -0.008555713994703569, -0.012677979325141312, -0.017340038148084016, -0.004532577903682724, 0.03188250044778791, 0.034895150720838805, 0.06302832567106627, 0.0644538110291535, -0.05013673655423878, 0.014593623709025684, -0.005317970307999118, 0.015197568389057765, 0.014701078079059108, 0.0540280715247069, -0.0454545454545455, 0.03301650825412698, 0.0, -0.09822309970384989, -0.0249145090376161, -0.007763640284666722, 0.006888068880688703, 0.016989806116330195, -0.017890295358649767, -0.020662075094423454, -0.0005802707930367725, -0.008314678605692294, -0.008858780614903661, -0.002829254491441445, 0.0046978161503301065, 0.008793820558526634, 0.0017580872011252425, -0.018494055482166365, 0.0, -0.00

In [13]:
pl.plot_return_distributions(rounded_returns)

In [14]:
avg_returns = a.calculate_average_return(rounded_returns)
median_returns = a.calculate_median_return(rounded_returns)
win_rate = a.calculate_win_rate(rounded_returns)

In [15]:
pl.plot_original_stats(avg_returns, median_returns, win_rate)

In [21]:
from scipy.stats import ks_2samp, anderson_ksamp

num_samples = len(rounded_returns[1]) if len(rounded_returns[1]) > 100 else 1000
num_samples = 1000

bootstrap_returns = a.bootstrap_sample(rounded_returns, n_samples=num_samples)
kde_returns = a.kde_sample(rounded_returns, n_samples=num_samples)
bayesian_returns = a.bayesian_sample(rounded_returns, n_samples=num_samples)
mixed_returns = a.mixed_sample(rounded_returns, n_samples=num_samples)

Processing day 1


Auto-assigning NUTS sampler...
Initializing NUTS using jitter+adapt_diag...
Multiprocess sampling (4 chains in 4 jobs)
NUTS: [nu, sigma, mu]


Sampling 4 chains for 1_500 tune and 1_000 draw iterations (6_000 + 4_000 draws total) took 1 seconds.
Sampling: [likelihood]


Processing day 2


Auto-assigning NUTS sampler...
Initializing NUTS using jitter+adapt_diag...
Multiprocess sampling (4 chains in 4 jobs)
NUTS: [nu, sigma, mu]


...

Auto-assigning NUTS sampler...
Initializing NUTS using jitter+adapt_diag...
Multiprocess sampling (4 chains in 4 jobs)
NUTS: [nu, sigma, mu]


Sampling 4 chains for 1_500 tune and 1_000 draw iterations (6_000 + 4_000 draws total) took 1 seconds.
Sampling: [likelihood]


Processing day 10


Auto-assigning NUTS sampler...
Initializing NUTS using jitter+adapt_diag...
Multiprocess sampling (4 chains in 4 jobs)
NUTS: [nu, sigma, mu]


Sampling 4 chains for 1_500 tune and 1_000 draw iterations (6_000 + 4_000 draws total) took 1 seconds.
Sampling: [likelihood]


In [22]:
results = []

for day in range(1,11):
    original_day = rounded_returns[day]
    bootstrap_day = bootstrap_returns[day]
    kde_day = kde_returns[day]
    bayesian_day = bayesian_returns[day]
    mixed_day = mixed_returns[day]

    # K-S Test
    ks_bootstrap = ks_2samp(original_day, bootstrap_day).pvalue
    ks_kde = ks_2samp(original_day, kde_day).pvalue
    ks_bayesian = ks_2samp(original_day, bayesian_day).pvalue
    ks_mixed = ks_2samp(original_day, mixed_day).pvalue
    # Anderson-Darling Test
    ad_bootstrap = anderson_ksamp([original_day, bootstrap_day]).significance_level
    ad_kde = anderson_ksamp([original_day, kde_day]).significance_level
    ad_bayesian = anderson_ksamp([original_day, bayesian_day]).significance_level
    ad_mixed = anderson_ksamp([original_day, mixed_day]).significance_level

    results.append({
        "day": day,
        "ks_bootstrap": ks_bootstrap,
        "ks_kde": ks_kde,
        "ks_bayesian": ks_bayesian,
        "ks_mixed": ks_mixed,
        "ad_bootstrap": ad_bootstrap,
        "ad_kde": ad_kde,
        "ad_bayesian": ad_bayesian,
        "ad_mixed": ad_mixed
    })

print(results)

[{'day': 1, 'ks_bootstrap': 0.9999999735864641, 'ks_kde': 0.9719408344447097, 'ks_bayesian': 0.8358072811825915, 'ks_mixed': 0.9700959236217366, 'ad_bootstrap': 0.25, 'ad_kde': 0.25, 'ad_bayesian': 0.25, 'ad_mixed': 0.25}, {'day': 2, 'ks_bootstrap': 0.997953736691308, 'ks_kde': 0.8659548435171026, 'ks_bayesian': 0.6567022121816789, 'ks_mixed': 0.9977667304382581, 'ad_bootstrap': 0.25, 'ad_kde': 0.25, 'ad_bayesian': 0.25, 'ad_mixed': 0.25}, {'day': 3, 'ks_bootstrap': 0.999999936855137, 'ks_kde': 0.7288567568540061, 'ks_bayesian': 0.8737918221642585, 'ks_mixed': 0.999996821250177, 'ad_bootstrap': 0.25, 'ad_kde': 0.25, 'ad_bayesian': 0.25, 'ad_mixed': 0.25}, {'day': 4, 'ks_bootstrap': 0.9999999997680262, 'ks_kde': 0.72929235986094, 'ks_bayesian': 0.8117835247427383, 'ks_mixed': 0.9940603835564504, 'ad_bootstrap': 0.25, 'ad_kde': 0.25, 'ad_bayesian': 0.25, 'ad_mixed': 0.25}, {'day': 5, 'ks_bootstrap': 1.0, 'ks_kde': 0.9998212771114277, 'ks_bayesian': 0.3238164260577887, 'ks_mixed': 0.99998

/var/folders/gr/66_s17nj0rq3hvgr_rys0w7m0000gn/T/ipykernel_12006/297188851.py:16: UserWarning:

p-value capped: true value larger than 0.25. Consider specifying `method` (e.g. `method=stats.PermutationMethod()`.)

/var/folders/gr/66_s17nj0rq3hvgr_rys0w7m0000gn/T/ipykernel_12006/297188851.py:17: UserWarning:

p-value capped: true value larger than 0.25. Consider specifying `method` (e.g. `method=stats.PermutationMethod()`.)

/var/folders/gr/66_s17nj0rq3hvgr_rys0w7m0000gn/T/ipykernel_12006/297188851.py:18: UserWarning:

p-value capped: true value larger than 0.25. Consider specifying `method` (e.g. `method=stats.PermutationMethod()`.)

/var/folders/gr/66_s17nj0rq3hvgr_rys0w7m0000gn/T/ipykernel_12006/297188851.py:19: UserWarning:

p-value capped: true value larger than 0.25. Consider specifying `method` (e.g. `method=stats.PermutationMethod()`.)



In [23]:
pl.qq_plot(rounded_returns, kde_returns)

In [24]:
pl.qq_plot(rounded_returns, bayesian_returns)

In [25]:
pl.qq_plot(rounded_returns, bootstrap_returns)

In [26]:
pl.qq_plot(rounded_returns, mixed_returns)

In [27]:
pl.plot_return_distributions(kde_returns)

In [28]:
pl.plot_return_distributions(bayesian_returns)

In [29]:
pl.plot_return_distributions(mixed_returns)

In [16]:
original_returns = a.generate_random_returns(df, dim_sample=500, n_iterations=1000)

Starting calculating returns...
Generated 1 samples.
Generated 2 samples.
Generated 3 samples.
Generated 4 samples.
Generated 5 samples.
Generated 6 samples.
Generated 7 samples.
Generated 8 samples.
Generated 9 samples.
Generated 10 samples.
Generated 11 samples.
Generated 12 samples.
Generated 13 samples.
Generated 14 samples.
Generated 15 samples.
Generated 16 samples.
Generated 17 samples.
Generated 18 samples.
Generated 19 samples.
Generated 20 samples.
Generated 21 samples.
Generated 22 samples.
Generated 23 samples.
Generated 24 samples.
Generated 25 samples.
Generated 26 samples.
Generated 27 samples.
Generated 28 samples.
Generated 29 samples.
Generated 30 samples.
Generated 31 samples.
Generated 32 samples.
Generated 33 samples.
Generated 34 samples.
Generated 35 samples.
Generated 36 samples.
Generated 37 samples.
Generated 38 samples.
Generated 39 samples.
Generated 40 samples.
Generated 41 samples.
Generated 42 samples.
Generated 43 samples.
Generated 44 samples.
Generated

In [17]:
results = a.calculate_metrics(original_returns)

In [22]:
pl.plot_metric_distributions(results, metric_name='kurtosis_return')